# 🟨 Gold Layer — Tabela `GoldYellow`
Este notebook cria a tabela **`GoldYellow`** a partir da camada Silver (`SilverYellow`),  
derivando colunas de **data/hora**, aplicando **tipagens**, e calculando KPIs com **window functions**:
- `Media_Passageiros` → média de passageiros por *Data_Viagem + Hora_Inicio_Viagem*  
- `Media_Valor_Total_Mensal` → média global de `total_amount` no conjunto (todas as linhas retornadas)

---


In [0]:
%sql
-- Contexto do catálogo/esquema onde a tabela será criada
USE CATALOG ifood_case;     -- catálogo lógico (workspace-level)
USE SCHEMA  nytaxi;         -- schema (database) de destino

-- Cria ou substitui a TABELA 'GoldYellow' materializando o resultado do SELECT
CREATE OR REPLACE TABLE GoldYellow AS 

-- =========================================
-- CTE 1: Ajustes de data/hora e tipos (GoldAdjustment)
-- - Deriva 'Data_Viagem' (string dd-MM-yyyy) e 'Hora_Inicio_Viagem' (int 0–23)
-- - Faz TRY_CAST dos campos numéricos para maior robustez
-- - Mantém campos de identificação e timestamps
-- =========================================
WITH GoldAdjustment AS (
  SELECT
      DATE_FORMAT(Data_Inicio_Viagem, 'dd-MM-yyyy') AS Data_Viagem           -- data de início da viagem no formato dd-MM-yyyy
    , TRY_CAST(Hora_Inicio_Viagem AS INT)           AS Hora_Inicio_Viagem    -- hora (0–23) como inteiro
    , Mes_Inicio_Viagem                                                      -- mês de início (1–12), já derivado na Silver
    , Mes_Fim_Viagem                                                         -- mês de fim   (1–12), já derivado na Silver
    , TRY_CAST(passenger_count AS DOUBLE)           AS passenger_count       -- quantidade de passageiros como double
    , vendorId                                                             -- id do fornecedor (empresa de táxi)
    , TRY_CAST(total_amount AS DOUBLE)              AS total_amount          -- valor total da corrida como double
    , tpep_dropoff_datetime                                                   -- timestamp de fim da corrida (original)
    , tpep_pickup_datetime                                                    -- timestamp de início da corrida (original)
  FROM 
    SilverYellow                                                             -- fonte: camada Silver (dados já limpos e padronizados)
),

-- =========================================
-- CTE 2: Cálculo de KPIs com Window Functions (DateTimeMediumPassenger)
-- - Media_Passageiros: média por Data_Viagem + Hora_Inicio_Viagem (partição por esses campos)
-- - Media_Valor_Total_Mensal: média global (OVER ()) no conjunto atual
--   Obs.: se quiser média por mês/partição específica, ajuste a cláusula OVER.
-- =========================================
DateTimeMediumPassenger AS (
  SELECT
      *
    , ROUND(
        AVG(passenger_count) OVER (PARTITION BY Data_Viagem, Hora_Inicio_Viagem)
      , 2) AS Media_Passageiros                                            -- média de passageiros por dia/hora
    , ROUND(
        AVG(total_amount)   OVER ()
      , 2) AS Media_Valor_Total_Mensal                                     -- média global de total_amount no conjunto
  FROM GoldAdjustment
)

-- =========================================
-- SELECT final
-- - Mantém KPIs calculados e campos de detalhe para auditoria/drill-down
-- - Sem GROUP BY: as janelas garantem que os KPIs sejam constantes dentro da partição
-- =========================================
SELECT
    Data_Viagem
  , Hora_Inicio_Viagem 
  , Media_Passageiros
  , Media_Valor_Total_Mensal
  , Mes_Fim_Viagem
  , Mes_Inicio_Viagem
  , passenger_count
  , total_amount
  , tpep_dropoff_datetime
  , tpep_pickup_datetime
  , vendorId
FROM 
  DateTimeMediumPassenger;


In [0]:
%sql

USE CATALOG ifood_case;
USE SCHEMA  nytaxi;

SELECT
  Mes_Inicio_Viagem               AS  Mes_Viagem
  ,  ROUND(AVG(total_amount), 2)  AS  Media_Mensal_Total_Amount
FROM 
  GoldYellow
WHERE
  Mes_Inicio_Viagem = Mes_Fim_Viagem
GROUP BY
  Mes_Viagem
ORDER BY
  Mes_Viagem ASC

In [0]:
%sql

USE CATALOG ifood_case;
USE SCHEMA  nytaxi;

WITH DateTimeMediumPassenger AS (
  SELECT
    DATE_FORMAT(tpep_pickup_datetime, 'dd-MM-yyyy')         AS  Data_Viagem
    , TRY_CAST(SUBSTRING(Hora_Inicio_Viagem, 1, 2) AS INT)  AS  Hora_Dia
    , TRY_CAST(passenger_count AS DOUBLE)                   AS  passenger_count
    , vendorId
    , total_amount
    , tpep_dropoff_datetime
    , tpep_pickup_datetime
  FROM
    GoldYellow
  WHERE
    Mes_Fim_Viagem        = 5
    AND Mes_Inicio_Viagem = 5
)
SELECT
  Data_Viagem
  , Hora_Dia
  , ROUND(AVG(passenger_count), 2)                        AS  Media_Passageiros
FROM
  DateTimeMediumPassenger
WHERE
  Hora_Dia BETWEEN 0 AND 23
GROUP BY
  Data_Viagem
  , Hora_Dia
ORDER BY
  Data_Viagem ASC,
  Hora_Dia ASC;

In [0]:
fim

In [0]:
from pyspark.sql.functions import split, explode, trim

df_exploded = df_validacao.select(
    explode(split('alerta_validacao', ';')).alias('alerta')
).withColumn('alerta', trim(F.col('alerta')))

# Remover linhas vazias
df_exploded = df_exploded.filter(F.col('alerta') != '')

# Contagem por tipo de alerta
df_alerta_count = df_exploded.groupBy('alerta').count().orderBy(F.desc('count'))
df_alerta_count.display()

alertas_pd = df_alerta_count.toPandas()
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.barh(alertas_pd['alerta'], alertas_pd['count'])
plt.xlabel('Ocorrências')
plt.title('Frequência dos Tipos de Alertas de Validação')
plt.gca().invert_yaxis()
plt.show()